In [2]:
%matplotlib inline
import numpy as np
import time
from random import random
import torch.nn as nn
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt
from util_data_pixels import *
from amorgibbs_v import *
from smc_v import *
from util_plots import *
from torch.distributions.dirichlet import Dirichlet
from torch.distributions.normal import Normal
from torchvision import datasets, transforms
import sys
import os
sys.path.append('/home/hao/Research/probtorch/')
from probtorch.util import expand_inputs
import probtorch
print('probtorch:', probtorch.__version__, 
      'torch:', torch.__version__, 
      'cuda:', torch.cuda.is_available())

probtorch: 0.0+5a2c637 torch: 0.5.0a0+3bb8c5e cuda: True


In [20]:
# model parameters
NUM_PIXELS = 64*64
NUM_HIDDEN = 256
NUM_LATENT = 2  

pixels=64
# training parameters
NUM_SAMPLES = 1
BATCH_SIZE = 128
NUM_EPOCHS = 100
LEARNING_RATE = 1e-3
BETA1 = 0.90
EPS = 1e-9
CUDA = torch.cuda.is_available()

# path parameters
MODEL_NAME = 'highvarprior'
DATA_PATH = './data'
WEIGHTS_PATH = 'experiment/'
RESTORE = False

ALPHA = 0.1
BETA = (2.0, 2.0, 1.0, 0.0, 1.0) 

In [21]:
pixels_path = '/home/hao/Research/amortized/AmortizedGibbs/'
ToGrayscaleTensor = transforms.Compose([transforms.Grayscale(), transforms.Resize(pixels), transforms.ToTensor()])
train_data = datasets.ImageFolder(root = pixels_path, transform=ToGrayscaleTensor)
train_loader = torch.utils.data.DataLoader(train_data, batch_size=BATCH_SIZE,shuffle=True)

In [22]:
class Encoder(nn.Module):
    def __init__(self, num_pixels=NUM_PIXELS, 
                       num_hidden=NUM_HIDDEN,
                       num_latent=NUM_LATENT):
        super(self.__class__, self).__init__()

        self.enc_hidden = nn.Sequential( 
                            nn.Linear(num_pixels, num_hidden),
                            nn.ReLU(),
                            nn.Linear(num_hidden, num_hidden),
                            nn.ReLU())
        
        self.z_mean = nn.Linear(num_hidden, num_latent)
        self.z_log_std = nn.Linear(num_hidden, num_latent)
    @expand_inputs
    def forward(self, images, labels=None, num_samples=NUM_SAMPLES):
        q = probtorch.Trace()
        hiddens = self.enc_hidden(images)
        q.normal(self.z_mean(hiddens),
                 self.z_log_std(hiddens).exp(),
                 name='z')
        return q

In [14]:
def binary_cross_entropy(x_mean, x, EPS=1e-9):
    return - (torch.log(x_mean + EPS) * x + 
              torch.log(1 - x_mean + EPS) * (1 - x)).sum(-1)

class Decoder(nn.Module):
    def __init__(self, num_pixels=NUM_PIXELS, 
                       num_hidden=NUM_HIDDEN,
                       num_latent=NUM_LATENT):
        super(self.__class__, self).__init__()
        
        self.dec_image = nn.Sequential(
                           nn.Linear(num_latent, num_hidden),
                           nn.ReLU(),
                           nn.Linear(num_hidden, num_pixels),
                           nn.Sigmoid())

    def forward(self, images, q=None, num_samples=NUM_SAMPLES, batch_size=BATCH_SIZE, num_latent=NUM_LATENT):
        p = probtorch.Trace()
        z_mean = torch.zeros(num_samples, batch_size, num_latent)
        z_std = torch.ones(num_samples, batch_size, num_latent)*210
        
        if CUDA:
            z_mean = z_mean.cuda()
            z_std = z_std.cuda()
        
        z = p.normal(z_mean, 
                     z_std,
                     value=q['z'],
                     name='z')
        
        images_mean = self.dec_image(z)
        p.loss(binary_cross_entropy, images_mean, images, name='images')
        
        return p

In [15]:
def elbo(q, p, alpha=ALPHA, beta=BETA, bias=1.0):
    return probtorch.objectives.marginal.elbo(q, p, sample_dim=0, batch_dim=1,
                                              alpha=alpha, beta=beta, bias=bias)

In [16]:
enc = Encoder()
dec = Decoder()
if CUDA:
    enc.cuda()
    dec.cuda()

optimizer =  torch.optim.Adam(list(enc.parameters())+list(dec.parameters()),
                              lr=LEARNING_RATE)

In [17]:
def train(data, enc, dec, optimizer):
    epoch_elbo = 0.0
    enc.train()
    dec.train()
    N = 0
    for b, (images, labels) in enumerate(data):
        if images.size()[0] == BATCH_SIZE:
            N += BATCH_SIZE
            images = images.view(-1, NUM_PIXELS)
            images = torch.where(images == 1.0, torch.FloatTensor([0.0]), torch.FloatTensor([1.0]))
            if CUDA:
                images = images.cuda()
            optimizer.zero_grad()
            q = enc(images, num_samples=NUM_SAMPLES)
            p = dec(images, q, num_samples=NUM_SAMPLES, batch_size=BATCH_SIZE)
            loss = -elbo(q, p)
            loss.backward()
            optimizer.step()
            if CUDA:
                loss = loss.cpu()
            epoch_elbo -= float(loss.item())
    return epoch_elbo / N

In [18]:
if not RESTORE:
    mask = {}
    for e in range(NUM_EPOCHS):
        train_start = time.time()
        train_elbo = train(train_loader, enc, dec, optimizer)
        train_end = time.time()
        print('[Epoch %d] Train: ELBO %.4e (%ds)' % (
                e, train_elbo, train_end - train_start))

    if not os.path.isdir(WEIGHTS_PATH):
        os.mkdir(WEIGHTS_PATH)
    torch.save(enc.state_dict(),
               '%s-pretrained-vae-enc' % MODEL_NAME)
    torch.save(dec.state_dict(),
               '%s-pretrained-vae-dec' % MODEL_NAME)

[Epoch 0] Train: ELBO -5.8635e+00 (5s)
[Epoch 1] Train: ELBO -2.9225e+00 (5s)
[Epoch 2] Train: ELBO -2.2846e+00 (5s)
[Epoch 3] Train: ELBO -1.8933e+00 (5s)
[Epoch 4] Train: ELBO -1.6135e+00 (6s)
[Epoch 5] Train: ELBO -1.4667e+00 (7s)
[Epoch 6] Train: ELBO -1.3730e+00 (5s)
[Epoch 7] Train: ELBO -1.3063e+00 (4s)
[Epoch 8] Train: ELBO -1.2728e+00 (4s)
[Epoch 9] Train: ELBO -1.2179e+00 (4s)
[Epoch 10] Train: ELBO -1.1810e+00 (4s)
[Epoch 11] Train: ELBO -1.1540e+00 (4s)
[Epoch 12] Train: ELBO -1.1200e+00 (4s)
[Epoch 13] Train: ELBO -1.0939e+00 (4s)
[Epoch 14] Train: ELBO -1.0758e+00 (4s)
[Epoch 15] Train: ELBO -1.0555e+00 (4s)
[Epoch 16] Train: ELBO -1.0370e+00 (4s)
[Epoch 17] Train: ELBO -1.0120e+00 (4s)
[Epoch 18] Train: ELBO -1.0007e+00 (4s)
[Epoch 19] Train: ELBO -9.8531e-01 (4s)
[Epoch 20] Train: ELBO -9.6491e-01 (4s)
[Epoch 21] Train: ELBO -9.5217e-01 (4s)
[Epoch 22] Train: ELBO -9.4131e-01 (4s)
[Epoch 23] Train: ELBO -9.2875e-01 (4s)
[Epoch 24] Train: ELBO -9.1821e-01 (4s)
[Epoch 25]

In [19]:
init_v = init_velocity(dt)
alpha_trans_0 = initial_trans_prior(K)
noise_cov = np.array([[1, 0], [0, 1]]) * noise_ratio
STATE, mu_ks, cov_ks, Pi, Y_true, A_true, Zs_true = generate_seq_T(T, K, dt, Boundary, init_v, noise_cov, radius)
generate_frames(STATE, Boundary, pixels, dpi, radius, 0)    
X_imgs = sample_pixels(T, pixels, 0).view(-1, NUM_PIXELS)
q = enc(X_imgs,num_samples=NUM_SAMPLES)
p = dec(X_imgs, q, num_samples=NUM_SAMPLES, batch_size=BATCH_SIZE)
z_means = q['z'].dist.mean.detach().squeeze(0).data.numpy()

coor = q['z'].value.squeeze(0)
Y = coor[1:] - coor[:-1]
cov_true = np.tile(noise_cov, (K, 1, 1))
dirs = np.array([[1, 1], [1, -1], [-1, -1], [-1, 1]])
mu_true = np.tile(np.absolute(init_v), (K, 1)) * dirs
plot_clusters(Y_true.data.numpy(), mu_true, cov_true, K)

NameError: name 'dt' is not defined